In [1]:
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.filterwarnings('ignore')
import unicodedata




In [2]:
# Aumentar número de columnas que se pueden ver
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.10f}'.format
# Cada columna será tan grande como sea necesario para mostrar todo su contenido
pd.set_option('display.max_colwidth', 0)

In [3]:
# Librerias necesarias para subir a Snowflake
import os
import json
import snowflake.connector 
#pip install snowflake-connector-python
from snowflake.connector.pandas_tools import write_pandas 
#pip install "snowflake-connector-python[pandas]"
from snowflake.snowpark import Session
from snowflake.connector import ProgrammingError
import PyPDF2
import io
import logging
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Paso 1: Definir la ruta al archivo JSON en el escritorio
desktop_path = "c:\\Users\\PGIC2\\OneDrive - PROCOLOMBIA\\Escritorio\\CREDENCIALES"
json_file_path = os.path.join(desktop_path, "CREDENCIAL.json")
json_file_path
 

'c:\\Users\\PGIC2\\OneDrive - PROCOLOMBIA\\Escritorio\\CREDENCIALES\\CREDENCIAL.json'

In [5]:

# Paso 2: Leer las credenciales desde el archivo JSON
with open(json_file_path, 'r') as file:
    credentials = json.load(file)
 
# Paso 3: Definir los parámetros de conexión usando las credenciales
connection_parameters = {
        "account": credentials["ACCOUNT_SNOWFLAKE"],
        "user": credentials["USER_SNOWFLAKE"],
        "password": credentials["PASSWORD_SNOWFLAKE"],
        "role": credentials["ROLE_SNOWFLAKE"],
        "warehouse": credentials["WAREHOUSE"]
    }
 
# Paso 5: Crear un objeto de conexión utilizando snowflake.connector
session = Session.builder.configs(connection_parameters).create()
print("Sesión actual:", {session})

Sesión actual: {<snowflake.snowpark.session.Session object at 0x000001A3945743D0>}


In [6]:
#Crear objeto de conexión llamado conn
conn=session.connection


In [7]:
# Crear un cursor para ejecutar consultas (No olvidar cerrar el curso y la conexión al terminar el proces)
cur = conn.cursor()
cur

In [8]:
# Asegurar que estamos en la ubicación que se desea para subir las bases de datos
cur.execute("SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;")
cur.fetchone()

('COMPUTE_WH', None, None)

In [9]:
def snow_sql(cursor, sql):
     #Ejecutar la consulta SQL
     cursor.execute(sql)
     # Obtener los resultados de la consulta
     column_names = [desc[0] for desc in cursor.description]
     results = cursor.fetchall()
    
     
     return results

In [10]:
query = snow_sql(cur,"""
ALTER USER Camilo SET RSA_PUBLIC_KEY='MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA4+ECm6dpwM7GkQWlNkmo
V58h34LaAgyjmIZpFHwqilqZzl2l/5S2doA1q3khM8vjGkkt+cnTgSvomyGU+43Q
qKubxWjnoErV/JqspBgKhdmtwx92ED1ph0NU24fcXZTw+6bdUsdzLCi6uUnZg+i0
R84LlLnA2QIsKOCiFafOT9Qh58hjKLVrJ3DSzb5Ne1O2VBC20Z4EFi0MPj8Nry6F
lb03K6i/64JMcs2AJm/z7NdRSU7mf2ynyJKyDccyPQ10nkbpLW+fIrKdOyTWC3Xd
e0m82BlwCZMrwqso407RGii7qhUTzaaE7DDMFPW5f6VUlMGeG9VgFCVof1r9ROqW
IwIDAQAB';         
""")

In [11]:
#query



In [12]:
query = snow_sql(cur,"""
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US'         
""")

In [13]:
query

[('Statement executed successfully.',)]

In [14]:
query = snow_sql(cur,"""
CREATE DATABASE IF NOT EXISTS cortex_search_tutorial_db           
""")

In [15]:
#query

In [16]:
query = snow_sql(cur,"""
USE DATABASE cortex_search_tutorial_db               
""")

In [17]:
#query

In [18]:
# Asegurar que estamos en la ubicación que se desea para subir las bases de datos
cur.execute("SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;")
cur.fetchone()

('COMPUTE_WH', 'CORTEX_SEARCH_TUTORIAL_DB', 'PUBLIC')

In [19]:
# Corrección de la consulta SQL
query = snow_sql(cur, """
    CREATE OR REPLACE STAGE cortex_search_tutorial_db.public.fomc
        DIRECTORY = (ENABLE = TRUE)
        ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
""")

In [20]:
#query

In [41]:
def upload_pdf_to_stage(connection, database, schema, stage, file_path):
    """
    Sube un archivo PDF a un stage en Snowflake con un nombre específico, sin compresión ni carpetas adicionales.

    Parámetros:
        connection (snowflake.connector.connection.SnowflakeConnection): Conexión activa de Snowflake.
        database (str): Nombre de la base de datos en Snowflake.
        schema (str): Nombre del esquema en Snowflake.
        stage (str): Nombre del stage en Snowflake.
        file_path (str): Ruta completa del archivo PDF a subir.
        snowflake_file_name (str): Nombre deseado para el archivo en Snowflake, sin extensión.

    Retorna:
        bool: True si la subida fue exitosa, False en caso de error.
    """
    try:
        # Verificar que el archivo sea un PDF
        if not file_path.lower().endswith('.pdf'):
            print("El archivo debe estar en formato PDF.")
            return False

        # Verificar que el archivo existe en el sistema de archivos
        if not os.path.exists(file_path):
            print(f"El archivo no existe en la ruta especificada: {file_path}")
            return False

        # Cambiar la base de datos y el esquema en Snowflake
        cur = connection.cursor()
        cur.execute(f"USE DATABASE {database};")
        cur.execute(f"USE SCHEMA {schema};")

        # Subir el archivo al stage con el nombre especificado, sin compresión
        stage_path = f"@{stage}"
        put_query = f"PUT 'file://{file_path}' {stage_path} AUTO_COMPRESS=FALSE OVERWRITE=TRUE"
        cur.execute(put_query)

        print(f"Archivo {file_path} subido exitosamente al stage '{stage}' como '{file_path}'.")
        return True

    except ProgrammingError as e:
        print(f"Error de programación en Snowflake: {e}")
        return False
    except Exception as e:
        print(f"Ocurrió un error: {e}")
        return False
    finally:
        cur.close()

In [42]:
file_path1 = "C://Users//PGIC2//OneDrive - PROCOLOMBIA//CAMILO ALARCON//PROYECTOS IA//DOCUMENTOS CRAISE//Code//tests//MANUAL_RTAS_PREG_FREC_PROCOLOMBIA_Final2.pdf"

In [43]:
file1 = upload_pdf_to_stage(conn, 'CORTEX_SEARCH_TUTORIAL_DB','PUBLIC','FOMC',file_path1)

Archivo C://Users//PGIC2//OneDrive - PROCOLOMBIA//CAMILO ALARCON//PROYECTOS IA//DOCUMENTOS CRAISE//Code//tests//MANUAL_RTAS_PREG_FREC_PROCOLOMBIA_Final2.pdf subido exitosamente al stage 'FOMC' como 'C://Users//PGIC2//OneDrive - PROCOLOMBIA//CAMILO ALARCON//PROYECTOS IA//DOCUMENTOS CRAISE//Code//tests//MANUAL_RTAS_PREG_FREC_PROCOLOMBIA_Final2.pdf'.


In [44]:
file_path2 = "C://Users//PGIC2//OneDrive - PROCOLOMBIA//CAMILO ALARCON//PROYECTOS IA//DOCUMENTOS CRAISE//Code//tests//sebsubsec.pdf"

In [45]:
file2 = upload_pdf_to_stage(conn, 'CORTEX_SEARCH_TUTORIAL_DB','PUBLIC','FOMC',file_path2)

Archivo C://Users//PGIC2//OneDrive - PROCOLOMBIA//CAMILO ALARCON//PROYECTOS IA//DOCUMENTOS CRAISE//Code//tests//sebsubsec.pdf subido exitosamente al stage 'FOMC' como 'C://Users//PGIC2//OneDrive - PROCOLOMBIA//CAMILO ALARCON//PROYECTOS IA//DOCUMENTOS CRAISE//Code//tests//sebsubsec.pdf'.


In [114]:
query = snow_sql(cur, """
    CREATE OR REPLACE FUNCTION cortex_search_tutorial_db.public.pdf_text_chunker(file_url STRING)
        RETURNS TABLE (chunk VARCHAR)
        LANGUAGE PYTHON
        RUNTIME_VERSION = '3.9'
        HANDLER = 'pdf_text_chunker'
        PACKAGES = ('snowflake-snowpark-python', 'PyPDF2', 'langchain')
        AS
$$
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from snowflake.snowpark.files import SnowflakeFile
import PyPDF2, io
import logging
import pandas as pd

class pdf_text_chunker:

    def read_pdf(self, file_url: str) -> str:
        logger = logging.getLogger("udf_logger")
        logger.info(f"Opening file {file_url}")

        with SnowflakeFile.open(file_url, 'rb') as f:
            buffer = io.BytesIO(f.readall())

        reader = PyPDF2.PdfReader(buffer)
        text = ""
        for page in reader.pages:
            try:
                text += page.extract_text().replace('\\n', ' ').replace('\\0', ' ')
            except:
                text = "Unable to Extract"
                logger.warn(f"Unable to extract from file {file_url}, page {page}")

        return text

    def process(self, file_url: str):
        text = self.read_pdf(file_url)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 2000,  # Ajuste del tamaño del chunk
            chunk_overlap = 300,  # Superposición para mantener el contexto
            length_function = len
        )

        chunks = text_splitter.split_text(text)
        df = pd.DataFrame(chunks, columns=['chunk'])

        yield from df.itertuples(index=False, name=None)
$$
""")



In [115]:
#query

In [116]:
# Corrección de la consulta SQL
query = snow_sql(cur, """
    CREATE OR REPLACE TABLE cortex_search_tutorial_db.public.docs_chunks_table AS
        SELECT
            relative_path,
            build_scoped_file_url(@cortex_search_tutorial_db.public.fomc, relative_path) AS file_url,
            -- preserve file title information by concatenating relative_path with the chunk
            CONCAT(relative_path, ': ', func.chunk) AS chunk,
            'Spanish' AS language
        FROM
            directory(@cortex_search_tutorial_db.public.fomc),
        
    TABLE(cortex_search_tutorial_db.public.pdf_text_chunker(build_scoped_file_url(@cortex_search_tutorial_db.public.fomc, relative_path))) AS func;
""")

In [117]:
#query

In [118]:
# Corrección de la consulta SQL
query = snow_sql(cur, """
    CREATE OR REPLACE CORTEX SEARCH SERVICE cortex_search_tutorial_db.public.fomc_meeting
        ON chunk
        ATTRIBUTES language
        WAREHOUSE = COMPUTE_WH
        TARGET_LAG = '1 hour'
        AS (
        SELECT
            chunk,
            relative_path,
            file_url,
            language
        FROM cortex_search_tutorial_db.public.docs_chunks_table
    );
""")

In [119]:
#query

In [120]:
query = snow_sql(cur, """
CREATE OR REPLACE TABLE CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.SEGUIMIENTO_ACTIVIDAD (
    ID NUMBER AUTOINCREMENT, 
    PREGUNTA VARCHAR, 
    RESPUESTA VARCHAR, 
    FECHA_HORA TIMESTAMP
)                 
""")

In [121]:
#query

In [122]:
# Asegurar que estamos en la ubicación que se desea para subir las bases de datos
cur.execute("SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;")
cur.fetchone()

('COMPUTE_WH', 'CORTEX_SEARCH_TUTORIAL_DB', 'PUBLIC')

In [123]:
query = snow_sql(cur, """
select * from CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOCS_CHUNKS_TABLE
""")  

In [124]:
query =session.sql("select * from CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.DOCS_CHUNKS_TABLE")

In [125]:
query.collect() 

[Row(RELATIVE_PATH='sebsubsec.pdf', FILE_URL='https://mp48210.us-east-2.aws.snowflakecomputing.com/api/files/01b8376a-0102-2dcc-0006-afce0020f7f6/28719349773/WiY7A8EHyKNn%2bAraqWkeuw3ZczTtX1WuVqXRTdzN1YVC4YnuewzoplFkzNzvLYzgGutsrl1kbSogfyiDb2SXJ%2fZIa8QpqojoyH63SjCpgyTmuRij8%2bHw3z7uSiDsE56dJBW0fNck6QrrSJRHvUaQpxtoziC%2fz2aMEgUpjSxnArCZiMvIZTY71NpsO3QkNY%2fxUFoA3PcFsjXer8fIiV8uLWYrhowIrZSg6y4lVBdulCV%2bZDa56zy9HaOH8Iy6PLtro6Jp3IxuJRx9%2f9zKiMB%2fzQ8Okw%3d%3d', CHUNK='sebsubsec.pdf: 1 2 3 4 5 6 7 8 9 10 11A B C D E F G Subpartida  ArancelariaPosicion Arancelaria SubPartida Tipo Cadena productiva Sector Subsector 0101101000Caballos  reproductores de  raza pura, vivosCaballos. asnos.  mulos y burdeganos.  vivos reproductores  de raza pura                                                                                                                                                                                                                                                               

In [ ]:
query =session.sql("select * from CORTEX_SEARCH_TUTORIAL_DB.PUBLIC.SEGUIMIENTO_ACTIVIDAD")  

In [ ]:
query.collect() 

[]

In [39]:
#cerrar las conexiones para ahorrar creditos
#conn.close()
#cur.close()
#session.close()